# Foundation of Computer Science - Final Project

Roll number: 846965

Name: Pranav

Surname: Kasela

Academic year: 2018/2019

Course: DATA SCIENCE

Faculty: DEPARTMENT OF INFORMATICS, SYSTEMS AND COMMUNICATION (DISCO)

Class: LM-91-Classe delle lauree magistrali in Tecniche e metodi per la societa dell'informazione

e-mail: p.kasela@campus.unimib.it

------------------------------

Structure of the folder

In [1]:
! ls

README.md               Solution_For_Exam.ipynb datasets


Dataset used and created by the code

In [2]:
! ls datasets/

Bridging_table.csv               googleplaystore.csv
Genre_Table.csv                  googleplaystore_user_reviews.csv


In [3]:
#Not needed for the exam but to calclute time of execution of the whole code
import time
start_time=time.time()

In [4]:
#Imported here all the libraries needed for this exam
import pandas as pd
import numpy as np
import re

In [5]:
#Uncomment to print all the DataFrame or Series if needed, we print all the columns
pd.set_option('display.max_columns', None)  # or any number instead of None
#pd.set_option('display.max_rows', None)  # or any number instead of None
#pd.set_option('display.max_colwidth', -1)  # or any number instead of -1

In [6]:
#I use it because I like seeing above the tables it's name.
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [7]:
#Read both datasets
googlestore = pd.read_csv("./datasets/googleplaystore.csv")
googlereviews = pd.read_csv("./datasets/googleplaystore_user_reviews.csv")

In [8]:
#The row 10472 is problematic, it doesn't have the Category camp so everything is shifted to the left
googlestore.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


Best choice for the row 10472 is the record removal

In [9]:
googlestore=googlestore.drop(10472)

The datasets structure:

In [10]:
display("googlestore.head()")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [11]:
display("googlereviews.head()")

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


## Exercise 1
1. Convert the app sizes to a number

In [12]:
#Checking the format of the data
googlestore.Size.unique()[0:5]

array(['19M', '14M', '8.7M', '25M', '2.8M'], dtype=object)

In [13]:
size_to_num = re.compile('(?P<number>\d+\.{0,1}\d*)(?P<prefix>\w*)')
#In this function we use lowercase k for kilo and uppercase M and G for Mega and Giga. 
#We Assume no file with any other unit otherwise the unit will be ignored.
def prefix_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000
    if unit == 'k':
        return 1000
    return 1

#This function can understand the format of the data given to it using regex,
#If the given data is not in the expected format it will return 'Not a Number'
#For example 'Varies with device' is transformed in NaN
def ConvertSizeToByte(Size):
    searched = size_to_num.search(Size)
    if searched is None:
        return np.nan
    else:
        prefix = searched.group('prefix')
        mult = prefix_to_mult(prefix)
        result = float(searched.group('number'))
        return int(result*mult)

We chose to create a new column for the converted data

In [14]:
googlestore['SizeInBytes'] = googlestore['Size'].apply(ConvertSizeToByte)

In [15]:
display("googlestore[['App','Size','SizeInBytes']].tail()") #Use tail because it shows a NaN

,App,Size,SizeInBytes
10836,Sya9a Maroc - FR,53M,53000000.0
10837,Fr. Mike Schmitz Audio Teachings,3.6M,3600000.0
10838,Parkinson Exercices FR,9.5M,9500000.0
10839,The SCP Foundation DB fr nn5n,Varies with device,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,19M,19000000.0


## Exercise 2
2. Convert the number of installs to a number

In [16]:
#Checking the format of the data
googlestore.Installs.unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

In [17]:
display("googlestore[googlestore.Installs=='0']")
#This is the only row with install not in the normal format (with the +)
#but it is not a problem since we will ignore the + while searching for the number

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device,NaN


In [18]:
install_to_num = re.compile('(?P<number>[\d,]+)') #the format is num,num,num,...,num

#We find using regex the number in the format 
def installToNumber(installs):
    found = install_to_num.search(installs)
    if(found):
        replacedComma = found.group('number').replace(',','') #replace the commas with nothing
        return int(replacedComma)
    else:
        return np.nan

In [19]:
googlestore['InstallNumber']=googlestore['Installs'].apply(installToNumber)

In [20]:
display("googlestore[['App','Installs','InstallNumber']].head()")

,App,Installs,InstallNumber
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000
3,Sketch - Draw & Paint,"50,000,000+",50000000
4,Pixel Draw - Number Art Coloring Book,"100,000+",100000


## Exercise 3
3. Transform “Varies with device” into a missing value

In [21]:
#replace 'Varies with device' with NaN.
googlestore.replace('Varies with device', np.nan,inplace=True)
#inplace is needed so the replace is executed on the dataframe itself

In [22]:
googlestore.tail()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,53000000.0,5000
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,3600000.0,100
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up,9500000.0,1000
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",NaN,NaN,NaN,1000
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",NaN,NaN,19000000.0,10000000


## Exercise 4
4. Convert Current Ver and Android Ver into dotted number (e.g. 4.0.3 or 4.2)

#### We start by converting the Android Ver

Due to the data format, we transform the Android version in 2 different column a minimum version and a maximum version considering the 'and up' value as the latest version 8.1 of android today.

In [23]:
#The versions with the W in the end are the wearable version we will not consider the W
googlestore['Android Ver'].value_counts()

4.1 and up       2451
4.0.3 and up     1501
4.0 and up       1375
4.4 and up        980
2.3 and up        652
5.0 and up        601
4.2 and up        394
2.3.3 and up      281
2.2 and up        244
4.3 and up        243
3.0 and up        241
2.1 and up        134
1.6 and up        116
6.0 and up         60
7.0 and up         42
3.2 and up         36
2.0 and up         32
5.1 and up         24
1.5 and up         20
4.4W and up        12
3.1 and up         10
2.0.1 and up        7
8.0 and up          6
7.1 and up          3
1.0 and up          2
4.0.3 - 7.1.1       2
5.0 - 8.0           2
5.0 - 7.1.1         1
5.0 - 6.0           1
7.0 - 7.1.1         1
4.1 - 7.1.1         1
2.2 - 7.1.1         1
Name: Android Ver, dtype: int64

In [24]:
print('There are {0} missing data values in Android Ver out of {1} more than 12%'
      .format(googlestore[googlestore['Android Ver'].isnull()].shape[0], googlestore.shape[0]))

There are 1364 missing data values in Android Ver out of 10840 more than 12%


The following cell contains the major_Android_ver considered 8.1 as of today.

We define two function one to compute the maximum version and one the minimum version.

In [25]:
MinVer_toNum=re.compile('^(?P<MinVer>\d+\.\d+\.{0,1}\d*)')
MaxVer_toNum=re.compile('(?P<MaxVer>\d+\.\d+\.{0,1}\d*)$')

#To be changed if new version is released, will be used instead of 'and up'
major_Android_ver = 8.1

#We Decide not to trow away the missing version since we would we throwing away more that 12% of the data!
def MinAndroidVersion_ToNum(version):
    if pd.isnull(version):
        return np.nan
    MinVer=MinVer_toNum.search(version)
    if MinVer is not None:
        return MinVer.group('MinVer')
    else:
        return np.nan
    
def MaxAndroidVersion_ToNum(version):
    #Since there are some NaN values
    if pd.isnull(version):
        return np.nan
    MaxVer=MaxVer_toNum.search(version)
    if MaxVer is not None:
        return MaxVer.group('MaxVer')
    else:
        return major_Android_ver

In [26]:
googlestore['Min_Android_Ver']=googlestore['Android Ver'].apply(MinAndroidVersion_ToNum)
googlestore['Max_Android_Ver']=googlestore['Android Ver'].apply(MaxAndroidVersion_ToNum)

In [27]:
display("googlestore[['App','Android Ver','Min_Android_Ver','Max_Android_Ver']].tail()")
#The varies with device becomes None

,App,Android Ver,Min_Android_Ver,Max_Android_Ver
10836,Sya9a Maroc - FR,4.1 and up,4.1,8.1
10837,Fr. Mike Schmitz Audio Teachings,4.1 and up,4.1,8.1
10838,Parkinson Exercices FR,2.2 and up,2.2,8.1
10839,The SCP Foundation DB fr nn5n,NaN,NaN,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,NaN,NaN,NaN


#### Here we do the Current Ver conversion

In [28]:
print('There are {0} missing data values in Android Ver out of {1} more than 12%'
      .format(googlestore[googlestore['Current Ver'].isnull()].shape[0], googlestore.shape[0]))

There are 1467 missing data values in Android Ver out of 10840 more than 12%


In [29]:
googlestore['Current Ver'].value_counts()

1.0                        809
1.1                        264
1.2                        178
2.0                        151
1.3                        145
1.0.0                      136
1.0.1                      119
1.4                         88
1.5                         81
1.0.2                       80
1.6                         65
1.0.3                       62
2.1                         61
3.0                         59
1.0.4                       58
1.7                         53
2.0.0                       51
1.0.5                       50
1.1.0                       49
1.1.1                       48
1.0.6                       47
1.2.1                       46
1.2.0                       43
1.8                         42
4.0                         40
1.9                         37
2.3.2                       35
2.4                         34
1.0.7                       34
1.0.9                       33
                          ... 
20180620                     1
3.0.2.0 

In [30]:
#It will search for any string with the format num.num.num...
#It should end with a number, can also not have a dot for example a new app can have version 1 and 
#not necessarily 1.0
currentVer = re.compile('(?P<ver>[\d+\.]*\d)') 

def convertCurrentVer(version):
    #Since we checked that there are some NaN values
    if pd.isnull(version):
        return np.nan
    CurVer=currentVer.search(version)
    if CurVer is not None:
        return CurVer.group('ver')
    else:
        return np.nan

In [31]:
googlestore['CurrentVersion']=googlestore['Current Ver'].apply(convertCurrentVer)

In [32]:
display("googlestore[['App','Current Ver','CurrentVersion']].head(8000).tail()")

,App,Current Ver,CurrentVersion
7995,Cartoon Wars: Blade,1.1.0,1.1.0
7996,FANDOM: Arrow and The Flash,2.9.8,2.9.8
7997,CW Beacon for Ham Radio,KM,NaN
7998,IZ2UUF Morse Koch CW,1.0.013,1.0.013
7999,Morse Decoder for Ham Radio,1.52,1.52


In [33]:
#checking why some non null Current Ver were converted to None
googlestore[googlestore.CurrentVersion.isnull() & ~googlestore['Current Ver'].isnull()][['App','Current Ver','CurrentVersion']]

,App,Current Ver,CurrentVersion
1423,Self Healing,Public.Heal,NaN
2501,HHS,Initial,NaN
3611,Latest Emmanuella Comedy Video,Natalia Studio Development,NaN
5218,AI Image Recognizer (beta),closed,NaN
5289,Ak-47 Wallpapers 2018,newversion,NaN
5981,BC iptv player,Final,NaN
7140,Cádiz CB Gades,opciÃ³n de cerrar,NaN
7509,App for Craigslist Pro - Buy & Sell Postings,App copyright,NaN
7510,Pro App for Craigslist,Copyright,NaN
7766,CR & CoC Private Server - Clash Barbarians PRO,Gratis,NaN


## Exercise 5
5. Remove the duplicates

We sort the dataset according to the App and number of reviews in case of the same app in descending order, and then we we drop the duplicates keeping only the first value so the one the one with more reviews.

In [34]:
googlestore=googlestore.sort_values(['App','Reviews'],ascending=False).drop_duplicates('App',keep='first')

In [35]:
#There is not more any duplicate
googlestore[googlestore.duplicated('App')]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver,CurrentVersion


In [36]:
#We Reset the index such as the dropped values are retaken and this way we don't have gaps
#in the index, it's just a personal choice not necessary.
googlestore=googlestore.reset_index().drop('index',axis = 1)
#And we name our index AppId, it will be needed later in exercise 8
googlestore.index.name="AppId"

In [37]:
display("googlestore.head()")
#The app with AppId 3 has a Price of 399.99$!

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver,CurrentVersion
AppId,,,,,,,,,,,,,,,,,,
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3 and up,4000000.0,1000000,4.0.3,8.1,1.1.3.2
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,Tools,"October 21, 2017",1.0,4.2 and up,3200000.0,10000,4.2,8.1,1.0
2,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,Social,"July 24, 2018",4.2.4,4.0.3 and up,18000000.0,1000000,4.0.3,8.1,4.2.4
3,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4 and up,26000000.0,10000,4.4,8.1,1.0.0
4,"뽕티비 - 개인방송, 인터넷방송, BJ방송",VIDEO_PLAYERS,NaN,414,59M,"100,000+",Free,0,Mature 17+,Video Players & Editors,"July 18, 2018",4.0.7,4.0.3 and up,59000000.0,100000,4.0.3,8.1,4.0.7


## Exercise 6
6. For each category, compute the number of apps

We have Removed the duplicated apps so all we need to do is do a groupby with category and use the function size(). Or we could have used the function value_counts() for the Series googlestore['Category']

In [38]:
googlestore.groupby('Category').size()
#googlestore.Category.value_counts() #Analogous as above

Category
ART_AND_DESIGN           61
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     222
BUSINESS                420
COMICS                   56
COMMUNICATION           315
DATING                  171
EDUCATION               108
ENTERTAINMENT            87
EVENTS                   64
FAMILY                 1876
FINANCE                 345
FOOD_AND_DRINK          112
GAME                    943
HEALTH_AND_FITNESS      288
HOUSE_AND_HOME           73
LIBRARIES_AND_DEMO       84
LIFESTYLE               369
MAPS_AND_NAVIGATION     131
MEDICAL                 395
NEWS_AND_MAGAZINES      254
PARENTING                60
PERSONALIZATION         376
PHOTOGRAPHY             281
PRODUCTIVITY            374
SHOPPING                202
SOCIAL                  239
SPORTS                  325
TOOLS                   828
TRAVEL_AND_LOCAL        219
VIDEO_PLAYERS           164
WEATHER                  79
dtype: int64

## Exercise 7
7. For each category, compute the average rating

In [39]:
#Dataframe is easier to read.
pd.DataFrame(googlestore.groupby('Category')['Rating'].mean())

,Rating
Category,
ART_AND_DESIGN,4.359322
AUTO_AND_VEHICLES,4.190411
BEAUTY,4.278571
BOOKS_AND_REFERENCE,4.344970
BUSINESS,4.098479
COMICS,4.181481
COMMUNICATION,4.121484
DATING,3.970149
EDUCATION,4.358879


## Exercise 8
8. Create two dataframes: one for the genres and one bridging apps and genders. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

#### We start by creating the Genre table from the googlestore dataframe

In [40]:
#We can have genres like Art & Design;Creativity so we need to separate them by splitting them first
#then we convert it to a DataFrame so the arrays within the array are separated and then we append them
genre = pd.DataFrame([genres.split(';') for genres in googlestore.Genres])
display("genre.head()")

,0,1
0,Entertainment,None
1,Tools,None
2,Social,None
3,Lifestyle,None
4,Video Players & Editors,None


In [41]:
#we can have a maximum of 2 genre per app but we assume not knowing it and use a more general approch
genre_series = pd.Series() #Empty Series
Apps2 = pd.Series() #Empty Series

for i in genre.columns:
    #just for convenience it indicates the column number of the genre dataframe created before
    genre_i = genre[i] 
    #We append in a single series all the columns of the dataframe genre
    genre_series = genre_series.append(genre_i)
    #We append as many times as genre also the application, we will need to later for the join table
    Apps2 = Apps2.append(googlestore.App)
#We drop the None values assuming all the apps to hava a genre and keep only the unique values
#So no app can have a None Genre.
genre_unique = genre_series.dropna().unique()

In [42]:
genre_table = pd.DataFrame(genre_unique).reset_index()

In [43]:
#Just to create nice index so we can export the csv file with GenreId
genre_table.columns = ['GenreId','Genre']

In [44]:
#esthetic choise to make id start from 1 and not 0
genre_table.GenreId = [x+1 for x in genre_table.GenreId]
#Now we have a genre_table
display("genre_table.head()")

,GenreId,Genre
0,1,Entertainment
1,2,Tools
2,3,Social
3,4,Lifestyle
4,5,Video Players & Editors


In [45]:
#Needed just for the print, will restore it as it was after the next cell
#Even though it could we left like it is. Couldn't find a way to print a dataframe without it's index
genre_table.set_index('GenreId',inplace=True)
genre_table.head()

,Genre
GenreId,
1,Entertainment
2,Tools
3,Social
4,Lifestyle
5,Video Players & Editors


In [46]:
#We set the index = None not to have the left index column in the .csv file
genre_table.to_csv("./datasets/Genre_Table.csv")

In [47]:
#I create a dictionary from Apps2 and genre_series and convert them into a dataframe
temp_table = pd.DataFrame({'Apps':Apps2,'Genre': genre_series})

In [48]:
#We drop the null values and reset the index
temp_table = temp_table.dropna()
temp_table = temp_table.reset_index()
#checking which column do we have
temp_table.columns

Index(['index', 'Apps', 'Genre'], dtype='object')

In [49]:
#We rename it just so all the tables have the same name when we need to merge them
temp_table.columns = ['AppId', 'App','Genre']

In [50]:
#I sort it so the same app with more genres are nearby (just esthetics)
bridging_table = pd.merge(temp_table,genre_table.reset_index(), on = 'Genre').sort_values('AppId')
display("bridging_table.head()")

,AppId,App,Genre,GenreId
0,0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment,1
592,1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools,2
1420,2,"💘 WhatsLov: Smileys of love, stickers and GIF",Social,3
1659,3,💎 I'm rich,Lifestyle,4
2029,4,"뽕티비 - 개인방송, 인터넷방송, BJ방송",Video Players & Editors,5


In [51]:
bridging_table[bridging_table.duplicated('AppId',keep=False)].sort_values('AppId').head(6)

,AppId,App,Genre,GenreId
2830,157,iTooch Mathématiques CP,Education,7
2260,157,iTooch Mathématiques CP,Education,7
2831,277,codeSpark Academy & The Foos,Education,7
6088,277,codeSpark Academy & The Foos,Educational,25
2832,353,Zowi App,Education,7
2271,353,Zowi App,Education,7


#### We see that the Genre Education is somehow duplicated in different Apps such as Zowi App, iTooch Mathématiques CP, etc. that is because of the format of googlestore dataframe, in the Genres column it has two times Education!

In [52]:
display("googlestore.iloc[[157,353]]")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver,CurrentVersion
AppId,,,,,,,,,,,,,,,,,,
157,iTooch Mathématiques CP,FAMILY,4.3,1018,35M,"100,000+",Free,0,Everyone,Education;Education,"June 28, 2017",4.6.2,4.0.3 and up,35000000.0,100000,4.0.3,8.1,4.6.2
353,Zowi App,FAMILY,4.5,516,21M,"50,000+",Free,0,Everyone,Education;Education,"December 21, 2016",1.4.1,4.0.3 and up,21000000.0,50000,4.0.3,8.1,1.4.1


#### It would create a problem since we were thinking of using this column as a composite primary key formed with AppId and GenreId, best thing is to remove the duplicated rows:

In [53]:
bridging_table.drop_duplicates(inplace=True) #keeps the first and removes only if same value for each column

In [54]:
bridging_table.duplicated().any()

False

In [55]:
#In the bridging table we don't need the app name we use AppId a better key and the same for Genre
bridging_table=bridging_table.reset_index().drop(['App','Genre','index'],axis = 1)
display("bridging_table.head()")

,AppId,GenreId
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5


In [56]:
bridging_table.to_csv("./datasets/Bridging_table.csv",index=None)

In [57]:
display("bridging_table","genre_table","googlestore[['App','Genres']]")

,AppId,GenreId
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
5,5,6
6,6,7
7,7,6
8,8,8
9,9,9


In [58]:
genre_table.reset_index(inplace=True)

#### Let's check if the merge works on the three tables

In [59]:
merge1 = bridging_table.merge(genre_table, on='GenreId')

In [60]:
merge1.head()

,AppId,GenreId,Genre
0,0,1,Entertainment
1,16,1,Entertainment
2,19,1,Entertainment
3,20,1,Entertainment
4,45,1,Entertainment


In [61]:
merge2 = merge1.merge(googlestore[['App','Genres']], on='AppId')

In [62]:
merge2.head()

,AppId,GenreId,Genre,App,Genres
0,0,1,Entertainment,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment
1,16,1,Entertainment,【Miku AR Camera】Mikuture,Entertainment
2,19,1,Entertainment,বাংলাflix,Entertainment
3,20,1,Entertainment,বাংলা টিভি প্রো BD Bangla TV,Entertainment
4,45,1,Entertainment,virtual lover 3D,Entertainment


#### Checking if we have 2 rows for the app Pixel Draw - Number Art Coloring Book once for Art & Design, once for Creativity

In [63]:
merge2[merge2.App=='Pixel Draw - Number Art Coloring Book']

,AppId,GenreId,Genre,App,Genres
4284,2636,15,Art & Design,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity
4285,2636,51,Creativity,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity


In [64]:
#We check on the bridging table too
bridging_table[bridging_table.AppId==2636] #The app with AppId 2636 is the Pixel Draw one.

,AppId,GenreId
2732,2636,51
2733,2636,15


## Exercise 9
9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [65]:
#We sort it because we want the column of the genre ordered so it's easier to find them
genre_unique_sorted = np.sort(genre_unique)

#### We create a new column for each element in genre_unique_sorted and assign it a vector of False of size the number of elements of the googlestore dataframe, and we will assign True later only where needed (only a few elements will have True value and all the rest will have False)

In [66]:
false_vector = np.repeat([False],googlestore.shape[0])
for elem in genre_unique_sorted:
    googlestore[elem]=false_vector

#Checking dtype of one of the columns
googlestore[genre_unique_sorted[0]].dtype

dtype('bool')

In [67]:
googlestore.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver,CurrentVersion,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,Business,Card,Casino,Casual,Comics,Communication,Creativity,Dating,Education,Educational,Entertainment,Events,Finance,Food & Drink,Health & Fitness,House & Home,Libraries & Demo,Lifestyle,Maps & Navigation,Medical,Music,Music & Audio,Music & Video,News & Magazines,Parenting,Personalization,Photography,Pretend Play,Productivity,Puzzle,Racing,Role Playing,Shopping,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
AppId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3 and up,4000000.0,1000000,4.0.3,8.1,1.1.3.2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,Tools,"October 21, 2017",1.0,4.2 and up,3200000.0,10000,4.2,8.1,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,Social,"July 24, 2018",4.2.4,4.0.3 and up,18000000.0,1000000,4.0.3,8.1,4.2.4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4 and up,26000000.0,10000,4.4,8.1,1.0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,"뽕티비 - 개인방송, 인터넷방송, BJ방송",VIDEO_PLAYERS,NaN,414,59M,"100,000+",Free,0,Mature 17+,Video Players & Editors,"July 18, 2018",4.0.7,4.0.3 and up,59000000.0,100000,4.0.3,8.1,4.0.7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [68]:
%%time
#We zip the only two important attributes for the exercise

#We use zip instead of iterating because pandas is slow when it comes 
#to iterate over the rows using a for cycle and the loc or iloc function
for i in zip(googlestore.index, googlestore.Genres):
    genrePerApp=i[1].split(';')
    for elem in genrePerApp:
        googlestore.at[i[0],elem] = True #We assign True only where needed and leave the rest False
        #googlestore.loc[i[0],elem] = True
        
#I changed from .loc to .at because I read on the internet and verified that at is faster than loc
#because 'loc' select multiple rows and/or columns while 'at' only a single value at a time. 
#The time goes from ~6 seconds with loc to ~100ms a huge difference on my system

CPU times: user 139 ms, sys: 3.17 ms, total: 143 ms
Wall time: 142 ms


#### Another way was using the ```pd.DataFrame.apply()``` on the dataframe with a the following function:

```python
def checkGenre(row,AppGenres):
    AppGenres=AppGenres.split(';')
    for genre in AppGenres:
        row[genre]=True
    return row

googlestore=googlestore.apply(lambda x: checkGenre(x,x['Genres']), axis = 1)
```

but it was slower than the method above, it took around 1-1.5 seconds on my system

In [69]:
googlestore.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Min_Android_Ver,Max_Android_Ver,CurrentVersion,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,Business,Card,Casino,Casual,Comics,Communication,Creativity,Dating,Education,Educational,Entertainment,Events,Finance,Food & Drink,Health & Fitness,House & Home,Libraries & Demo,Lifestyle,Maps & Navigation,Medical,Music,Music & Audio,Music & Video,News & Magazines,Parenting,Personalization,Photography,Pretend Play,Productivity,Puzzle,Racing,Role Playing,Shopping,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
AppId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3 and up,4000000.0,1000000,4.0.3,8.1,1.1.3.2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,Tools,"October 21, 2017",1.0,4.2 and up,3200000.0,10000,4.2,8.1,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,Social,"July 24, 2018",4.2.4,4.0.3 and up,18000000.0,1000000,4.0.3,8.1,4.2.4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4 and up,26000000.0,10000,4.4,8.1,1.0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,"뽕티비 - 개인방송, 인터넷방송, BJ방송",VIDEO_PLAYERS,NaN,414,59M,"100,000+",Free,0,Mature 17+,Video Players & Editors,"July 18, 2018",4.0.7,4.0.3 and up,59000000.0,100000,4.0.3,8.1,4.0.7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


## Exercise 10
10. For each genre, compute the average rating. What is the genre with highest average?

In [70]:
googlestore['Rating'].isna().any() #We have missing value in the Rating column

True

In [71]:
#Since we created the new columns for each genre, let's use them. Could have also used a groupby on each column
average_rating_per_genre=pd.Series() #Empty series, we use series because later we need use idxmax() 
for genere in genre_unique:
    ratings=googlestore[googlestore[genere]==True]['Rating']
    average_rating_per_genre[genere]=ratings.mean(skipna=True)#Skip the nan values when computing the mean
    #The mean is by deafault with skipna=True, but we put it explicitly to show the need here.

In [72]:
#I convert it into a Dataframe so it's easier to read.
pd.DataFrame(average_rating_per_genre,columns=['average_rating'])

,average_rating
Entertainment,4.093028
Tools,4.039917
Social,4.247291
Lifestyle,4.094040
Video Players & Editors,4.044667
Comics,4.181481
Education,4.290381
Books & Reference,4.343275
Communication,4.121790
Maps & Navigation,4.036441


In [73]:
#The maximum is
print("'{}' is the genre with highest ratings with {} average rating"
            .format(average_rating_per_genre.idxmax(),
            average_rating_per_genre[average_rating_per_genre.idxmax()]))

'Events' is the genre with highest ratings with 4.435555555555555 average rating


## Exercise 11
11. For each app, compute the approximate income, obtain as a product of number of installs and price.

In [74]:
#First we need to convert from Price to numbers
money_to_float = re.compile('\D*(?P<amount>\d+\.{0,1}\d*)')

def Convert(money):
    found = money_to_float.search(money)
    if found:
        return float(found.group('amount'))
    else:
        return None


googlestore['PriceInNumber']=googlestore.Price.apply(Convert)

In [75]:
#We create a new column with the approximate income from the store
#InstallNumber was already converted into a number in the exercise 2
googlestore['approximate_income']=googlestore.InstallNumber * googlestore.PriceInNumber

In [76]:
googlestore[['App','Price','InstallNumber','PriceInNumber','approximate_income']][googlestore.Price!='0'].head()

,App,Price,InstallNumber,PriceInNumber,approximate_income
AppId,,,,,
3,💎 I'm rich,$399.99,10000,399.99,3999900.00
27,Билеты ПДД CD 2019 PRO,$1.49,100,1.49,149.00
32,Égalité et Réconciliation,$2.99,500,2.99,1495.00
35,¡Ay Caramba!,$1.99,1,1.99,1.99
41,weather HD,$1.99,10000,1.99,19900.00


## Exercise 12
12. For each app, compute its minimum and maximum Sentiment_polarity

In [77]:
googlereviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [78]:
googlereviews.isna().any() 
#It can be seen by pure forture also in the precedent cell, 
#but generally we need to use this line to check for null values

App                       False
Translated_Review          True
Sentiment                  True
Sentiment_Polarity         True
Sentiment_Subjectivity     True
dtype: bool

#### There are some NaN values but it should not bother us because we use the min and max function of groupby which uses the value NaN only if there is no other value.

#### I convert the following into a DataFrame just so it's easier to read and then it's easier to merge them.

In [79]:
minSentiment = pd.DataFrame(googlereviews.groupby('App')['Sentiment_Polarity'].min())

In [80]:
maxSentiment = pd.DataFrame(googlereviews.groupby('App')['Sentiment_Polarity'].max())

In [81]:
MinMaxSentiment = minSentiment.merge(maxSentiment, on='App',suffixes=['_min','_max'])

#### We show only the ones without NaN, because the only onr useful

In [82]:
MinMaxSentiment[MinMaxSentiment.notnull().any(axis=1)]

,Sentiment_Polarity_min,Sentiment_Polarity_max
App,,
10 Best Foods for You,-0.800000,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500,0.910000
11st,-1.000000,1.000000
1800 Contacts - Lens Store,-0.300000,0.838542
1LINE – One Line with One Touch,-0.825000,1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000,1.000000
21-Day Meditation Experience,-0.265625,0.587500
"2Date Dating App, Love and matching",-0.645833,1.000000
2GIS: directory & navigator,-0.375000,1.000000


In [83]:
print("Program executed in %s seconds" % (time.time()-start_time))

Program executed in 2.3806872367858887 seconds
